# Import Dependencies

In [3]:
# import libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/faisal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data

## Collection

In [2]:
# Collect data using kaggle api
! kaggle competitions download -c fake-news
! unzip fake-news.zip

100%|██████████████████████████████████████| 46.5M/46.5M [00:14<00:00, 3.22MB/s]
100%|██████████████████████████████████████| 46.5M/46.5M [00:14<00:00, 3.29MB/s]
Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [5]:
news_dataset_train = pd.read_csv("train.csv")
news_dataset_test = pd.read_csv("test.csv")

In [6]:
# shape of the dataset

news_dataset_train.shape

(20800, 5)

In [7]:
# viewing first few rows of the dataset

news_dataset_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
# checking for missing values

news_dataset_train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
# replacing null values with empty string

news_dataset_train = news_dataset_train.fillna("")

In [11]:
# combine title and author
news_dataset_train['content'] = news_dataset_train['title'] + ' ' + news_dataset_train['author']

In [12]:
print(news_dataset_train['content'])

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2        Why the Truth Might Get You Fired Consortiumne...
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799              What Keeps the F-35 Alive David Swanson
Name: content, Length: 20800, dtype: object


In [13]:
# Separating features and label

X = news_dataset_train.drop("label", axis = 1)
Y = news_dataset_train["label"]

## Stemming

In [14]:
port_stem = PorterStemmer()

In [ ]:
# stemming function
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content